In [60]:
import numpy as np
from collections import defaultdict

In [61]:
def loadEpisodes(fichier):
    """ Loads episodes in arrays """
    
    # Load the file
    with open(fichier, 'r') as f:
        episodes = []
        for episode in f.readlines():                                           # Read lines one by one
            episode = np.array([p.split(':')                                    # Remove the last ';' and split':'
                                for p in episode[:-2].split(';')], float)
            episode = np.array(episode, int)
            episode = episode[episode[:,1].argsort()]                           # Sort the array in order of the 
            episodes.append(episode)                                            # infection time 
        
    return episodes       

In [62]:
class IC():
    def __init__(self, episodes, nbIteration=1):
        """ Algorithme IC (Independent Cascade)
            Setting up the inference mechanism and the learning algorithm of
            infections probabilities.
        """
        
        self.episodes = episodes                                                # Array of episodes
        self.nbIteration = nbIteration                                          # Nb Iterations for reaching convergence
        self.nbUser = max([max(epi[:,0]) for epi in self.episodes]) + 1         # Nomber of users or distincts nodes
        self.predecessors = defaultdict(dict)                                   # Dictionary of predecessors for each user
        self.successors = defaultdict(dict)                                     # Dictionary of successors for each user
        self.dMoins = np.zeros((self.nbUser, self.nbUser))                      # Set of episodes D-
        self.dPlus = {(i,j):[] for i in range(0,self.nbUser)                    # Set of episodes D+
                      for j in range(0, self.nbUser)}   
        self.theta = np.array([[ np.random.random()                             # Probability of the precedent step ô
                                for i in range(0, self.nbUser)]             
                               for j in range(0, self.nbUser)])
        
    
    def createGraph(self):
        """ Creates the graph (tree) of episodes"""
        
        for episode in self.episodes:
            listeSuccessors = [episode[episode[:,1] > episode[i,1]][:,0]        # List of list of successors for each user
                                for i in range(len(episode))]   
            for i, successeur in enumerate(listeSuccessors):                    # for the list of successors of each user
                for v in successeur:                                            # for every successor of a user
                    u, proba = episode[i,0], np.random.random()                 # Generate a probability so within (0,1)
                    self.successors[u][v] = proba                               # u ---(proba)---> v 
                    self.predecessors[v][u] = proba                             # v ---(proba)---> u

        
    def ptD(self):
        """ Estimates each success probability """
        
        p = dict()
        for d, episode in enumerate(self.episodes):
            users, tempsU = episode[:,0], np.unique(episode[:,1])               # List of users of an episode and distinct
            p[d] = np.ones(self.nbUser)                                         # time
            for t in range(1, len(tempsU)):                                           # For each time tU of the episode D
                ptd, hasPred = 1., False
                for u, user in enumerate(users):
                    predU = episode[episode[:,1] < tempsU[t]][:,0]              # List of predecessors of user u at time tU
                    for v in predU:
                        if v in self.predecessors[user]:
                            ptd *= (1 - self.theta[v][user])
                            hasPred = True
                    if hasPred:
                        p[d][u] = 1-ptd
        return p
                    
        
    
    def setOfdPlus(self):
        """ This method fills the set of episodes D+ which satisfies
            both u € D(t) and v € D(>t) """
        
        for d, episode in enumerate(self.episodes):
            for i in range(0,len(episode)):
                for j in range(0,len(episode)):
                    u, v = episode[i][0], episode[j][0]
                    tu, tv = episode[i][1], episode[j][1]
                    if u != v and tv > tu:                                      # If it's not the same user and v is infected
                        self.dPlus[u,v].append(d)                               # after u, then add the episode to the set
                        
    
    def setOfdMoins(self):
        """ This method fills the set of episodes D- which satisfies
            both u € D(t) and v not € D(t) """
        
        for episode in self.episodes:
            for i in range(0,len(episode)):
                for j in range(0, self.nbUser):
                    if (j not in episode[:,0]):
                        self.dMoins[episode[i][0]][j] += 1
                        
    
    def fit(self):
        """ Estimates each diffusion probability """
        
        self.createGraph()
        self.setOfdPlus()
        self.setOfdMoins()
        
        for i in range(0, self.nbIteration):
            p = self.ptD()
            for u in range(0, self.nbUser):
                for v in range(0, self.nbUser):
                    sumThetaPtd = 0
                    sumDSets = len(self.dPlus[u,v]) + self.dMoins[u][v]
                    for d in self.dPlus[u,v]:
                        sumThetaPtd += self.theta[u][v]/p[d][v]
                    self.theta[u][v] = sumThetaPtd/sumDSets

In [63]:
ic = IC(loadEpisodes("cascades_train.txt"))
ic.fit()
print(ic.theta[1])

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in double_scalars


[ 0.05523535         nan  0.05408906  0.11084702  0.05900183  0.10486607
  0.05594093  0.00786007  0.01328483  0.0850231   0.0637469   0.10498114
  0.0773769   0.01342088  0.12632155  0.10783061  0.0514721   0.0334151
  0.08704321  0.05405921  0.00058558  0.00151584  0.17896522  0.06578457
  0.00920494  0.09418698  0.08699895  0.09485255  0.01397375  0.04040093
  0.06736768  0.08598154  0.00071239  0.04268263  0.06241718  0.04652047
  0.06027433  0.15081401  0.01824538  0.00485031  0.10264537  0.06836479
  0.16671521  0.01736387  0.01958326  0.00024252  0.03706491  0.01988232
  0.1177455   0.08615251  0.03210392  0.07246299  0.04998572  0.10789648
  0.02613847  0.01781894  0.03147888  0.0673931   0.08840583  0.05539892
  0.00246661  0.12333019  0.09969643  0.08902023  0.08723418  0.08798865
  0.1989361   0.1118617   0.11391862  0.10339978  0.11310786  0.12097764
  0.11898297  0.07896499  0.05966964  0.04297484  0.12932045  0.1251069
  0.0634086   0.11636987  0.10845749  0.10058585  0.0